In [1]:
!pip install transformers[sentencepiece]

In [2]:
!pip install transformers[torch]

In [3]:
import accelerate

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

In [5]:
from transformers import (AutoTokenizer,AutoConfig,AutoModelForSequenceClassification, DataCollatorWithPadding,TrainingArguments,Trainer)

In [4]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00


In [6]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [7]:
import torch

In [9]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [10]:
import evaluate
import numpy as np

In [11]:
#load imdb dataset
imdb_dataset= load_dataset("imdb")

#define subsample size
N=1000
rand_idx=np.random.randint(24999,size=N)

#Test and train data
x_train=imdb_dataset['train'][rand_idx]['text']
y_train=imdb_dataset['train'][rand_idx]['label']

x_test=imdb_dataset['test'][rand_idx]['text']
y_test=imdb_dataset['test'][rand_idx]['label']

#creating new dataset
dataset=DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

#loading dataset
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [12]:
#model
model_checkpoint='distilbert-base-uncased'

#define label maps
id2label={0:"Negative",1:"Positive"}
label2id={"Negative":0,"Positive":1}

#generate classification model from model checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2,id2label=id2label,label2id=label2id)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [13]:
#creating tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

#adding pad token
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
#creating tokenize function
def tokenize_function(examples):
  text=examples["text"]

  #tokenize and truncate text
  tokenizer.truncation_side="left"
  tokenized_inputs=tokenizer(text,return_tensors="np",truncation=True,max_length=512)
  return tokenized_inputs

In [15]:
#tokenize training and validation datasets
tokenized_dataset=dataset.map(tokenize_function,batched=True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [16]:
#create data collator
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
#accuracy evaluation metric
accuracy=evaluate.load("accuracy")

#creating evaluation function
def compute_metric(p):
  predictions, labels=p
  predictions=np.argmax(predictions,axis=1)
  return {"accuracy":accuracy.compute(predictions=predictions,references=labels)}

In [18]:
peft_config=LoraConfig(task_type="SEQ_CLS",r=4,lora_alpha=32,lora_dropout=0.01,target_modules=['q_lin'])
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [19]:
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [20]:
#hyperparameters
lr=1e-3
batch_size=4
num_epochs=10

In [21]:
#define_training_arguments
from transformers import TrainingArguments
training_args=TrainingArguments(output_dir=model_checkpoint+"-lora-text-classification",learning_rate=lr,per_device_train_batch_size=batch_size,per_device_eval_batch_size=batch_size,num_train_epochs=num_epochs,weight_decay=0.01,evaluation_strategy="epoch",save_strategy="epoch",load_best_model_at_end=True,)


In [ ]:
#creating trainer object
trainer=Trainer(model=model,args=training_args,train_dataset=tokenized_dataset["train"],eval_dataset=tokenized_dataset["validation"],tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metric)

#train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.359408,{'accuracy': 0.876}
2,0.435400,0.459222,{'accuracy': 0.872}
3,0.435400,0.581164,{'accuracy': 0.875}
4,0.190600,0.676465,{'accuracy': 0.882}
5,0.190600,0.944222,{'accuracy': 0.872}
6,0.082800,0.935228,{'accuracy': 0.876}
7,0.082800,0.981916,{'accuracy': 0.877}


Trainer is attempting to log a value of "{'accuracy': 0.876}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Trainer is attempting to log a value of "{'accuracy': 0.872}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.wa

In [1]:
model.to('cpu')
print("Trained model predictions:")
print("--------------------------")
for text in text_list:
  inputs=tokenizer.encode(text,return_tensor="pt").to("cpu")
  logits=model(inputs).logits
  predictions=torch.max(logits,1).indices
  print(text+"-"+id2label[predictions.tolist()[0]])

NameError: name 'model' is not defined